In [ ]:
# Install TF, TFH 
!pip install tf-nightly
!pip install -q -U tf-hub-nightly

In [ ]:
# Import libraries

import matplotlib.pylab as plt
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers
import numpy as np
import PIL.Image as Image
import os
import pathlib

In [ ]:
# Download Image Classification model

classifier_url ="https://tfhub.dev/google/tf2-preview/mobilenet_v2/classification/2"

In [ ]:
# Set Classifier

IMAGE_SHAPE = (224, 224)

classifier = tf.keras.Sequential([
    hub.KerasLayer(classifier_url, input_shape=IMAGE_SHAPE+(3,))
])

In [ ]:
# Download test image

grace_hopper = tf.keras.utils.get_file('image.jpg','https://storage.googleapis.com/download.tensorflow.org/example_images/grace_hopper.jpg')
grace_hopper = Image.open(grace_hopper).resize(IMAGE_SHAPE)
grace_hopper

In [ ]:
# Add batch dimension

grace_hopper = np.array(grace_hopper)/255.0
grace_hopper.shape

In [ ]:
# Run model : result as 1001 element vecotr of logits, rating probability of each class

result = classifier.predict(grace_hopper[np.newaxis, ...])
result.shape

In [ ]:
# Find prediction

predicted_class = np.argmax(result[0], axis=-1)
predicted_class

In [ ]:
# Download ImageNet labels

labels_path = tf.keras.utils.get_file('ImageNetLabels.txt','https://storage.googleapis.com/download.tensorflow.org/data/ImageNetLabels.txt')
imagenet_labels = np.array(open(labels_path).read().splitlines())

In [ ]:
# Decode prediction

plt.imshow(grace_hopper)
plt.axis('off')
predicted_class_name = imagenet_labels[predicted_class]
_ = plt.title("Prediction: " + predicted_class_name.title())

In [ ]:
# Export model

import time

t = time.time()

export_path = "/content/saved_models/{}".format(int(t))
classifier.save(export_path)

export_path

In [ ]:
# Convert SavedModel file to TFL file

converter = tf.lite.TFLiteConverter.from_saved_model(export_path)
tflite_model = converter.convert()

# Save model

with open('model_default.tflite', 'wb') as f:
  f.write(tflite_model)

# Print size of model

p = pathlib.Path('model_default.tflite')
p.write_bytes(tflite_model)

13976588

In [ ]:
# Download model

from google.colab import files
files.download("model_default.tflite")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Convert to Dynamic range quantization model (DRQ)

converter = tf.lite.TFLiteConverter.from_saved_model(export_path)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_quant_model = converter.convert()

# Save model

with open('model_drq.tflite', 'wb') as f:
  f.write(tflite_model)

# Print size of model

p = pathlib.Path('model_drq.tflite')
p.write_bytes(tflite_quant_model)

3932240

In [ ]:
# Download model

from google.colab import files
files.download("model_drq.tflite")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Convert to Float16 quantization model (F16Q)

converter = tf.lite.TFLiteConverter.from_saved_model(export_path)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_types = [tf.float16]
tflite_quant_model = converter.convert()

# Save model

with open('model_f16q.tflite', 'wb') as f:
  f.write(tflite_model)

# Print size of model

p = pathlib.Path('model_f16q.tflite')
p.write_bytes(tflite_quant_model)

7010912

In [ ]:
# Download model

from google.colab import files
files.download("model_f16q.tflite")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Convert to Float fallback integer quantization (FFIQ)

# Load reperesenative dataset
repdata = tf.keras.datasets.cifar10
(train_images, train_labels), (test_images, test_labels) = repdata.load_data()

# Normalize the input image so that each pixel value is between 0 to 1.
train_images = train_images.astype(np.float32) / 255.0
test_images = test_images.astype(np.float32) / 255.0

def representative_dataset_gen():
  for input in tf.data.Dataset.from_tensor_slices(train_images).batch(1).take(100):
    yield [input]

converter = tf.lite.TFLiteConverter.from_saved_model(export_path)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_dataset_gen
tflite_quant_model = converter.convert()

with open('model_ffiq.tflite', 'wb') as f:
  f.write(tflite_model)

In [ ]:
# Convert to Full integer quantization model (FIQ)

# Load reperesenative dataset
repdata = tf.keras.datasets.cifar10
(train_images, train_labels), (test_images, test_labels) = repdata.load_data()

# Normalize the input image so that each pixel value is between 0 to 1.
train_images = train_images.astype(np.float32) / 255.0
test_images = test_images.astype(np.float32) / 255.0

def representative_dataset_gen():
  for input in tf.data.Dataset.from_tensor_slices(train_images).batch(1).take(100):
    yield [input]

converter = tf.lite.TFLiteConverter.from_saved_model(export_path)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_dataset_gen
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]

converter.inference_input_type = tf.uint8
converter.inference_output_type = tf.uint8
tflite_quant_model = converter.convert()

with open('model_drq.tflite', 'wb') as f:
  f.write(tflite_model)